## Setup: Required Imports & Configuration

In [1]:
# Essential imports
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 200)

print("✓ All required libraries imported successfully")

✓ All required libraries imported successfully


## Load Data (From CSV or Raw Excel)

**Option A:** Load the processed CSV (if you've already run the full pipeline)

**Option B:** Load raw Excel and run quick pipeline

In [2]:
# OPTION A: Load pre-processed CSV (fastest)
csv_path = r'd:\DataViz_project\social_media_data_with_niches.csv'
try:
    feat_df = pd.read_csv(csv_path)
    feat_df['Post Timestamp Parsed'] = pd.to_datetime(feat_df['Post Timestamp Parsed'])
    print(f"✓ Loaded processed CSV: {len(feat_df):,} rows")
    data_loaded = True
except FileNotFoundError:
    print("⚠ CSV not found. Will use OPTION B (load raw Excel)")
    data_loaded = False

✓ Loaded processed CSV: 97,159 rows


In [3]:
# OPTION B: Load raw Excel and run quick ETL pipeline (uncomment if needed)
if not data_loaded:
    import kagglehub
    
    # Download dataset
    path = kagglehub.dataset_download("aliredaelblgihy/social-media-engagement-report")
    raw_path = path + "/social_media_engagement_data.xlsx"
    
    # Quick ETL pipeline
    def parse_timestamp(val):
        if pd.isna(val):
            return pd.NaT
        for fmt in (None, '%Y-%m-%d %H:%M:%S', '%d/%m/%Y %H:%M:%S', '%m/%d/%Y %H:%M:%S'):
            try:
                return pd.to_datetime(val, format=fmt) if fmt else pd.to_datetime(val)
            except:
                continue
        try:
            base = pd.Timestamp('2024-01-01')
            return base + pd.to_timedelta(str(val).replace(' ', ''))
        except:
            return pd.NaT
    
    # Load and clean
    raw_df = pd.read_excel(raw_path)
    raw_df['Post Timestamp Parsed'] = raw_df['Post Timestamp'].apply(parse_timestamp)
    
    # Type conversion and cleaning
    num_cols = ['Likes', 'Comments', 'Shares', 'Impressions', 'Reach', 'Engagement Rate', 'Audience Age']
    for c in num_cols:
        if c in raw_df.columns:
            raw_df[c] = pd.to_numeric(raw_df[c], errors='coerce')
    
    raw_df = raw_df.drop_duplicates(subset=['Post ID'])
    raw_df = raw_df[(raw_df[['Likes', 'Comments', 'Shares', 'Impressions', 'Reach']] >= 0).all(axis=1)]
    
    # Feature engineering
    ts = raw_df['Post Timestamp Parsed']
    raw_df['Year'] = ts.dt.year
    raw_df['Month'] = ts.dt.month
    raw_df['Day'] = ts.dt.day
    raw_df['Hour'] = ts.dt.hour
    raw_df['Weekday'] = ts.dt.day_name()
    raw_df['Weekend'] = raw_df['Weekday'].isin(['Saturday', 'Sunday'])
    
    # KPI calculations
    with np.errstate(divide='ignore', invalid='ignore'):
        raw_df['True Engagement Rate'] = (raw_df['Likes'] + raw_df['Comments'] + raw_df['Shares']) / raw_df['Reach'] * 100
        raw_df['Interaction Rate'] = (raw_df['Comments'] + raw_df['Shares']) / raw_df['Impressions'] * 100
        raw_df['Share Amplification'] = raw_df['Shares'] / raw_df['Reach'] * 100
        raw_df['Exposure Efficiency'] = raw_df['Reach'] / raw_df['Impressions'] * 100
    
    raw_df = raw_df.replace([np.inf, -np.inf], np.nan)
    
    # Virality score
    raw_df['z_Interaction Rate'] = stats.zscore(raw_df['Interaction Rate'].fillna(raw_df['Interaction Rate'].median()))
    raw_df['z_Share Amplification'] = stats.zscore(raw_df['Share Amplification'].fillna(raw_df['Share Amplification'].median()))
    raw_df['Virality Score v2'] = raw_df['z_Interaction Rate'] * 0.6 + raw_df['z_Share Amplification'] * 0.4
    
    # Content niche classification
    niche_keywords = {
        'Politics': ['vote', 'election', 'government', 'political', 'campaign', 'congress', 'president'],
        'Business': ['business', 'company', 'market', 'economy', 'investment', 'finance', 'corporate'],
        'Technology': ['tech', 'software', 'digital', 'innovation', 'ai', 'computer', 'data'],
        'Health': ['health', 'medical', 'wellness', 'fitness', 'doctor', 'hospital', 'disease'],
        'Sports': ['sport', 'game', 'player', 'team', 'match', 'athlete', 'competition'],
        'Entertainment': ['movie', 'music', 'celebrity', 'show', 'entertainment', 'film', 'artist'],
        'Education': ['education', 'school', 'student', 'learning', 'university', 'teacher'],
        'Travel': ['travel', 'trip', 'destination', 'vacation', 'tourism', 'journey', 'explore'],
        'Food': ['food', 'recipe', 'restaurant', 'cooking', 'meal', 'cuisine', 'chef'],
        'Fashion': ['fashion', 'style', 'clothing', 'outfit', 'trend', 'designer', 'wardrobe'],
        'Lifestyle': ['life', 'lifestyle', 'living', 'home', 'family', 'daily', 'routine'],
        'Social Issues': ['community', 'society', 'social', 'people', 'public', 'citizen', 'rights'],
        'Environment': ['environment', 'climate', 'nature', 'green', 'sustainable', 'eco', 'planet'],
        'Legal': ['law', 'legal', 'court', 'lawyer', 'attorney', 'justice', 'regulation']
    }
    
    def classify_niche(text):
        if pd.isna(text) or not isinstance(text, str):
            return 'Uncategorized'
        text_lower = text.lower()
        scores = {niche: sum(1 for kw in keywords if kw in text_lower) 
                  for niche, keywords in niche_keywords.items()}
        scores = {k: v for k, v in scores.items() if v > 0}
        return max(scores.items(), key=lambda x: x[1])[0] if scores else 'General'
    
    raw_df['Content Niche'] = raw_df['Post Content'].apply(classify_niche)
    feat_df = raw_df
    print(f"✓ ETL pipeline complete: {len(feat_df):,} rows processed")

## Data Quality Check

In [4]:
print("📊 Dataset Overview:")
print(f"  Total Posts: {len(feat_df):,}")
print(f"\n📱 Platform Distribution:")
print(feat_df['Platform'].value_counts())
print(f"\n📝 Content Niches:")
print(feat_df['Content Niche'].value_counts().head(10))
print(f"\n✅ Data ready for visualization!")

📊 Dataset Overview:
  Total Posts: 97,159

📱 Platform Distribution:
Platform
Twitter      24460
LinkedIn     24434
Facebook     24166
Instagram    24099
Name: count, dtype: int64

📝 Content Niches:
Content Niche
General          56141
Technology       12957
Politics          4155
Environment       3571
Entertainment     2909
Social Issues     2649
Business          2597
Education         2367
Health            2222
Lifestyle         1930
Name: count, dtype: int64

✅ Data ready for visualization!


---
# 15 BUSINESS VISUALIZATIONS
---

## 1️⃣ Platform Performance Comparison - Multi-Metric Radar
**Question:** Which platform should we prioritize for maximum ROI?

In [5]:
platform_metrics = feat_df.groupby('Platform').agg({
    'True Engagement Rate': 'median',
    'Interaction Rate': 'median',
    'Share Amplification': 'median',
    'Exposure Efficiency': 'median',
    'Reach': 'mean'
}).round(2)

scaler = MinMaxScaler(feature_range=(0, 100))
platform_metrics_scaled = pd.DataFrame(
    scaler.fit_transform(platform_metrics),
    columns=platform_metrics.columns,
    index=platform_metrics.index
)

fig = go.Figure()
for platform in platform_metrics_scaled.index:
    fig.add_trace(go.Scatterpolar(
        r=platform_metrics_scaled.loc[platform].values,
        theta=platform_metrics_scaled.columns,
        fill='toself',
        name=platform
    ))

fig.update_layout(
    polar=dict(radialaxis=dict(visible=True, range=[0, 100])),
    showlegend=True,
    title='1. Platform Performance Across Key Metrics (Normalized 0-100)',
    height=600
)
fig.show()

## 2️⃣ Best Posting Schedule - Weekly Heatmaps
**Question:** When should we schedule posts for maximum engagement?

In [6]:
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
platform_schedule = feat_df.groupby(['Platform', 'Weekday', 'Hour'])['True Engagement Rate'].median().reset_index()

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Instagram', 'Facebook', 'Twitter', 'LinkedIn'),
    vertical_spacing=0.12,
    horizontal_spacing=0.1
)

platforms = ['Instagram', 'Facebook', 'Twitter', 'LinkedIn']
positions = [(1,1), (1,2), (2,1), (2,2)]

for platform, pos in zip(platforms, positions):
    data = platform_schedule[platform_schedule['Platform'] == platform]
    pivot = data.pivot(index='Weekday', columns='Hour', values='True Engagement Rate')
    pivot = pivot.reindex(weekday_order)
    
    fig.add_trace(
        go.Heatmap(
            z=pivot.values,
            x=pivot.columns,
            y=pivot.index,
            colorscale='RdYlGn',
            showscale=True if pos == (1,2) else False,
            hovertemplate='%{y}<br>Hour: %{x}<br>Engagement: %{z:.2f}%<extra></extra>'
        ),
        row=pos[0], col=pos[1]
    )

fig.update_layout(
    title='2. Best Posting Times by Platform (Median Engagement Rate)',
    height=800,
    showlegend=False
)
fig.update_xaxes(title_text="Hour of Day")
fig.update_yaxes(title_text="Day of Week")
fig.show()

## 3️⃣ Content Niche Performance by Platform
**Question:** What content topics work best on each platform?

In [7]:
niche_platform = feat_df.groupby(['Platform', 'Content Niche']).agg({
    'True Engagement Rate': 'median',
    'Post ID': 'count'
}).rename(columns={'Post ID': 'Post Count'}).reset_index()

niche_platform = niche_platform[niche_platform['Post Count'] >= 50]
top_niches = niche_platform.sort_values(['Platform', 'True Engagement Rate'], ascending=[True, False])
top_niches = top_niches.groupby('Platform').head(5)

fig = px.bar(
    top_niches,
    x='True Engagement Rate',
    y='Content Niche',
    color='Platform',
    facet_col='Platform',
    facet_col_wrap=2,
    title='3. Top 5 Content Niches by Platform',
    labels={'True Engagement Rate': 'Median Engagement Rate (%)'},
    height=800,
    text='True Engagement Rate'
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_yaxes(matches=None)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()

## 4️⃣ Post Type ROI Analysis
**Question:** Which post types give the best return on investment?

In [8]:
post_type_analysis = feat_df.groupby(['Platform', 'Post Type']).agg({
    'True Engagement Rate': ['median', 'mean'],
    'Reach': 'mean',
    'Virality Score v2': 'median',
    'Post ID': 'count'
}).round(2)
post_type_analysis.columns = ['Median_Engagement', 'Mean_Engagement', 'Avg_Reach', 'Median_Virality', 'Post_Count']
post_type_analysis = post_type_analysis.reset_index()

fig = px.scatter(
    post_type_analysis,
    x='Avg_Reach',
    y='Median_Engagement',
    size='Post_Count',
    color='Post Type',
    facet_col='Platform',
    facet_col_wrap=2,
    title='4. Post Type Performance: Engagement vs Reach (Bubble Size = Volume)',
    labels={'Avg_Reach': 'Average Reach', 'Median_Engagement': 'Median Engagement Rate (%)'},
    height=700,
    hover_data=['Post_Count', 'Median_Virality']
)
fig.update_traces(marker=dict(line=dict(width=1, color='white')))
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()

## 5️⃣ Weekend vs Weekday Performance
**Question:** Should we post differently on weekends?

In [9]:
weekend_comparison = feat_df.groupby(['Platform', 'Weekend']).agg({
    'True Engagement Rate': 'median',
    'Interaction Rate': 'median',
    'Share Amplification': 'median',
    'Post ID': 'count'
}).round(2).reset_index()
weekend_comparison['Day_Type'] = weekend_comparison['Weekend'].map({True: 'Weekend', False: 'Weekday'})

fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=('Engagement Rate', 'Interaction Rate', 'Share Amplification'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}, {'type': 'bar'}]]
)

metrics = ['True Engagement Rate', 'Interaction Rate', 'Share Amplification']
for idx, metric in enumerate(metrics, 1):
    for day_type in ['Weekday', 'Weekend']:
        data = weekend_comparison[weekend_comparison['Day_Type'] == day_type]
        fig.add_trace(
            go.Bar(
                x=data['Platform'],
                y=data[metric],
                name=day_type,
                showlegend=True if idx == 1 else False,
                text=data[metric],
                texttemplate='%{text:.1f}%'
            ),
            row=1, col=idx
        )

fig.update_layout(title='5. Weekend vs Weekday Performance', barmode='group', height=500)
fig.show()

## 6️⃣ Audience Demographics Impact
**Question:** Which audience segments engage most with our content?

In [10]:
demo_analysis = feat_df.groupby(['Platform', 'Audience Gender', pd.cut(feat_df['Audience Age'], bins=[0, 25, 35, 45, 55, 100], labels=['18-25', '26-35', '36-45', '46-55', '56+'])]).agg({
    'True Engagement Rate': 'median',
    'Post ID': 'count'
}).reset_index()
demo_analysis.columns = ['Platform', 'Gender', 'Age_Group', 'Median_Engagement', 'Post_Count']
demo_analysis = demo_analysis[demo_analysis['Post_Count'] >= 20]

fig = px.bar(
    demo_analysis,
    x='Age_Group',
    y='Median_Engagement',
    color='Gender',
    facet_col='Platform',
    facet_col_wrap=2,
    title='6. Engagement by Age Group and Gender',
    labels={'Median_Engagement': 'Median Engagement Rate (%)'},
    barmode='group',
    height=700,
    text='Median_Engagement'
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()

## 7️⃣ Virality Pattern Analysis
**Question:** What factors make content go viral?

In [11]:
viral_threshold = feat_df['Virality Score v2'].quantile(0.95)
viral_patterns = feat_df[feat_df['Virality Score v2'] >= viral_threshold].groupby(['Platform', 'Post Type', 'Content Niche']).agg({
    'Post ID': 'count',
    'Share Amplification': 'median'
}).reset_index()
viral_patterns = viral_patterns.sort_values('Post ID', ascending=False).head(20)

fig = px.treemap(
    viral_patterns,
    path=['Platform', 'Post Type', 'Content Niche'],
    values='Post ID',
    color='Share Amplification',
    color_continuous_scale='YlOrRd',
    title='7. Viral Content Patterns (Top 5% Posts)',
    labels={'Post ID': 'Viral Post Count', 'Share Amplification': 'Avg Share %'}
)
fig.update_layout(height=600)
fig.show()

## 8️⃣ Engagement Rate Distribution
**Question:** What's the typical engagement range we should expect?

In [12]:
fig = px.violin(
    feat_df,
    x='Platform',
    y='True Engagement Rate',
    color='Platform',
    box=True,
    points='outliers',
    title='8. Engagement Rate Distribution by Platform',
    labels={'True Engagement Rate': 'True Engagement Rate (%)'},
    height=500
)
fig.add_hline(y=feat_df['True Engagement Rate'].median(), line_dash="dash", 
              annotation_text=f"Overall Median: {feat_df['True Engagement Rate'].median():.2f}%")
fig.show()

## 9️⃣ Share-to-Like Ratio Analysis
**Question:** Which content drives sharing behavior vs passive liking?

In [13]:
feat_df['Share_to_Like_Ratio'] = (feat_df['Shares'] / (feat_df['Likes'] + 1)) * 100
share_analysis = feat_df.groupby(['Platform', 'Content Niche']).agg({
    'Share_to_Like_Ratio': 'median',
    'Post ID': 'count'
}).reset_index()
share_analysis = share_analysis[share_analysis['Post ID'] >= 30]
share_analysis = share_analysis.sort_values('Share_to_Like_Ratio', ascending=False).groupby('Platform').head(8)

fig = px.bar(
    share_analysis,
    x='Share_to_Like_Ratio',
    y='Content Niche',
    color='Platform',
    facet_col='Platform',
    facet_col_wrap=2,
    title='9. Share-to-Like Ratio by Niche (Higher = More Viral)',
    labels={'Share_to_Like_Ratio': 'Shares per 100 Likes'},
    height=700,
    orientation='h'
)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()

## 🔟 Monthly Engagement Trends
**Question:** Are engagement rates improving or declining over time?

In [14]:
monthly_trends = feat_df.groupby(['Platform', feat_df['Post Timestamp Parsed'].dt.to_period('M')]).agg({
    'True Engagement Rate': 'median',
    'Post ID': 'count'
}).reset_index()
monthly_trends['Month'] = monthly_trends['Post Timestamp Parsed'].astype(str)

fig = px.line(
    monthly_trends,
    x='Month',
    y='True Engagement Rate',
    color='Platform',
    markers=True,
    title='10. Monthly Engagement Trends by Platform',
    labels={'True Engagement Rate': 'Median Engagement Rate (%)'},
    height=500
)
fig.update_xaxes(tickangle=45)
fig.show()

## 1️⃣1️⃣ Geographic Performance Analysis
**Question:** Which locations drive the best engagement?

In [15]:
top_30_locations = feat_df['Audience Location'].value_counts().head(30).index
geo_data = feat_df[feat_df['Audience Location'].isin(top_30_locations)]
geo_performance = geo_data.groupby('Audience Location').agg({
    'True Engagement Rate': 'median',
    'Reach': 'mean',
    'Post ID': 'count'
}).reset_index().sort_values('True Engagement Rate', ascending=False)

fig = px.bar(
    geo_performance,
    x='True Engagement Rate',
    y='Audience Location',
    orientation='h',
    title='11. Top 30 Locations by Median Engagement Rate',
    labels={'True Engagement Rate': 'Median Engagement Rate (%)'},
    color='True Engagement Rate',
    color_continuous_scale='Viridis',
    height=800,
    text='True Engagement Rate'
)
fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig.show()

## 1️⃣2️⃣ Content Mix Optimizer
**Question:** What's the ideal ratio of post types for each platform?

In [16]:
content_mix = feat_df.groupby(['Platform', 'Post Type']).agg({
    'Post ID': 'count',
    'True Engagement Rate': 'median'
}).reset_index()
content_mix['Percentage'] = content_mix.groupby('Platform')['Post ID'].transform(lambda x: (x / x.sum()) * 100)

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Instagram', 'Facebook', 'Twitter', 'LinkedIn'),
    specs=[[{'type': 'pie'}, {'type': 'pie'}],
           [{'type': 'pie'}, {'type': 'pie'}]]
)

platforms = ['Instagram', 'Facebook', 'Twitter', 'LinkedIn']
positions = [(1,1), (1,2), (2,1), (2,2)]

for platform, pos in zip(platforms, positions):
    data = content_mix[content_mix['Platform'] == platform]
    fig.add_trace(
        go.Pie(
            labels=data['Post Type'],
            values=data['Percentage'],
            name=platform,
            textinfo='label+percent'
        ),
        row=pos[0], col=pos[1]
    )

fig.update_layout(title_text='12. Current Content Mix by Platform', height=700, showlegend=False)
fig.show()

## 1️⃣3️⃣ Peak Performance Hours
**Question:** What are the top 3 hours for each platform?

In [17]:
top_hours = feat_df.groupby(['Platform', 'Hour']).agg({
    'True Engagement Rate': 'median',
    'Post ID': 'count'
}).reset_index()
top_hours = top_hours.sort_values(['Platform', 'True Engagement Rate'], ascending=[True, False])
top_hours = top_hours.groupby('Platform').head(3)

fig = px.bar(
    top_hours,
    x='Hour',
    y='True Engagement Rate',
    color='Platform',
    facet_col='Platform',
    facet_col_wrap=2,
    title='13. Top 3 Posting Hours by Platform',
    labels={'True Engagement Rate': 'Median Engagement Rate (%)'},
    height=600,
    text='True Engagement Rate'
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()

## 1️⃣4️⃣ Engagement Funnel Analysis
**Question:** Where do we lose audience attention from Impression to Action?

In [18]:
funnel_data = feat_df.groupby('Platform').agg({
    'Impressions': 'mean',
    'Reach': 'mean',
    'Likes': 'mean',
    'Comments': 'mean',
    'Shares': 'mean'
}).reset_index()

platforms = ['Instagram', 'Facebook', 'Twitter', 'LinkedIn']
fig = make_subplots(
    rows=1, cols=4,
    subplot_titles=platforms,
    specs=[[{'type': 'funnel'}] * 4]
)

for idx, platform in enumerate(platforms, 1):
    data = funnel_data[funnel_data['Platform'] == platform].iloc[0]
    fig.add_trace(
        go.Funnel(
            y=['Impressions', 'Reach', 'Likes', 'Comments', 'Shares'],
            x=[data['Impressions'], data['Reach'], data['Likes'], data['Comments'], data['Shares']],
            textinfo="value+percent initial",
            name=platform
        ),
        row=1, col=idx
    )

fig.update_layout(title_text='14. Engagement Funnel: Impressions → Actions', height=500, showlegend=False)
fig.show()

## 1️⃣5️⃣ Competitive Benchmarking
**Question:** How do we compare to industry standards? (Percentile Analysis)

In [19]:
percentiles = [25, 50, 75, 90, 95]
benchmark_data = []
for platform in feat_df['Platform'].unique():
    platform_data = feat_df[feat_df['Platform'] == platform]
    for p in percentiles:
        benchmark_data.append({
            'Platform': platform,
            'Percentile': f'{p}th',
            'Engagement Rate': platform_data['True Engagement Rate'].quantile(p/100)
        })

benchmark_df = pd.DataFrame(benchmark_data)

fig = px.line(
    benchmark_df,
    x='Percentile',
    y='Engagement Rate',
    color='Platform',
    markers=True,
    title='15. Performance Benchmarks: Where Do You Stand?',
    labels={'Engagement Rate': 'True Engagement Rate (%)'},
    height=500
)
fig.add_annotation(
    text="💡 Tip: Aim for 75th percentile or higher for competitive advantage",
    xref="paper", yref="paper",
    x=0.5, y=-0.15,
    showarrow=False,
    font=dict(size=12)
)
fig.show()

---
## ✅ Analysis Complete!

### 📊 Next Steps
1. **Export these charts** for presentations or dashboards
2. **Use the insights** to optimize your content strategy
3. **Track metrics** monthly to measure improvement

### 🎯 Key Recommendations
- Identify your best performing platform from Viz #1
- Schedule posts using the times from Viz #2
- Focus on top niches from Viz #3
- Use post types with highest ROI from Viz #4
- Target your best demographic segments from Viz #6